In [ ]:
import os
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
from torch.utils.data import Dataset
import pandas as pd


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split

In [ ]:
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.read_csv('/content/drive/My Drive/deeplearning/final project/animedataset/anime-dataset-2023.csv')
df = df[['English name', 'Genres', 'Image URL']]
df['Genres'] = df['Genres'].apply(lambda x: x.split(', '))  # Split genre strings into lists

# One-hot encode the genres
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['Genres'])
genre_labels = mlb.classes_
df['encoded_genres'] = list(genres_encoded)

<ipython-input-5-1d72101b9018>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genres'] = df['Genres'].apply(lambda x: x.split(', '))  # Split genre strings into lists
<ipython-input-5-1d72101b9018>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['encoded_genres'] = list(genres_encoded)


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2)

In [ ]:
num_genres = len(df['encoded_genres'][0])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.dummy_image = torch.zeros(3, 224, 224)
        self.dummy_label = torch.zeros(num_genres)  # Adjust based on the number of genres

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_url = self.dataframe.iloc[idx]['Image URL']
        labels = self.dataframe.iloc[idx]['encoded_genres']
        labels = torch.tensor(labels, dtype=torch.float)
        try:
            response = requests.get(img_url)
            image = Image.open(BytesIO(response.content))

            # Convert grayscale images to RGB
            if image.mode != 'RGB':
                image = image.convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image, labels

        except (UnidentifiedImageError, requests.exceptions.RequestException):
            # Return dummy image and label in case of an error
            return self.dummy_image, self.dummy_label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Adjust size as per model requirement
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
from torch.utils.data import DataLoader

train_dataset = CustomDataset(train_df, transform=transform)
val_dataset = CustomDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # Adjust batch size as needed
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

In [ ]:
from sklearn.metrics import hamming_loss
import numpy as np

def calculate_hamming_loss(y_true, y_pred, threshold=0.5):
    # Applying a threshold to the predictions
    y_pred = (y_pred > threshold).float()

    # Converting tensors to numpy arrays
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()

    return hamming_loss(y_true, y_pred)

In [ ]:
from copy import deepcopy
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            all_labels = []
            all_predictions = []

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)

                # collect all labels
                all_labels.append(labels)
                all_predictions.append(outputs)

            all_labels = torch.cat(all_labels)
            all_predictions = torch.cat(all_predictions)
            epoch_hamming_loss = calculate_hamming_loss(all_labels, all_predictions)
            print(f'{phase} Hamming Loss: {epoch_hamming_loss:.4f} ')

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f}')

            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = deepcopy(model.state_dict())

        print()

    print('Training complete')
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_genres)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 186MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from torch.optim import lr_scheduler
criterion = nn.BCEWithLogitsLoss() # used for multi label classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
trained_model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 0/9
----------
train Hamming Loss: 0.0785 
train Loss: 0.2186
val Hamming Loss: 0.0807 
val Loss: 0.2242

Epoch 1/9
----------
train Hamming Loss: 0.0773 
train Loss: 0.2071
val Hamming Loss: 0.0781 
val Loss: 0.2085

Epoch 2/9
----------
train Hamming Loss: 0.0770 
train Loss: 0.2037
val Hamming Loss: 0.0787 
val Loss: 0.2101

Epoch 3/9
----------
train Hamming Loss: 0.0762 
train Loss: 0.1993
val Hamming Loss: 0.0773 
val Loss: 0.2047

Epoch 4/9
----------
train Hamming Loss: 0.0758 
train Loss: 0.1966
val Hamming Loss: 0.0774 
val Loss: 0.2099

Epoch 5/9
----------
train Hamming Loss: 0.0749 
train Loss: 0.1927
val Hamming Loss: 0.0776 
val Loss: 0.2059

Epoch 6/9
----------
train Hamming Loss: 0.0742 
train Loss: 0.1888
val Hamming Loss: 0.0766 
val Loss: 0.2027

Epoch 7/9
----------
train Hamming Loss: 0.0697 
train Loss: 0.1720
val Hamming Loss: 0.0749 
val Loss: 0.1982

Epoch 8/9
----------
train Hamming Loss: 0.0662 
train Loss: 0.1620
val Hamming Loss: 0.0754 
val Loss: 